<html>
<head>
<style>
h1 {text-align: center;}
p {text-align: center;}
div {text-align: center;}
</style>
</head>
<body>

<h1>Data Processing</h1>

<h3>Asbjørn Fyhn & Emil Kolko Beckett</h3>
<h3>Seminar: Topics in Sovereign Debt (Spring 2025)</h3>

</body>
</html>

In [4]:
import os
import pandas as pd 
import numpy as np
import warnings
from datetime import datetime
# pretty print
from pprint import pprint
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

path_to_data = r'/Users/asbjornfyhn/Desktop/Seminar - sovergin debt/code/data/'
path_to_raw_data = path_to_data + 'raw/'


# Table of Contents:
* [External debt to GDP](#External_Debt)

# Loading in data 

# External Debt and GDP data <span id="External_Debt"></span> 


In [5]:
#-------------- Find the external debt to GDP ratio
df_ex_debt = pd.read_excel(path_to_raw_data+r'ext_debt.xlsx',)
df_ex_debt = df_ex_debt.set_index(['Country Name','Country Code'],).iloc[:,2:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'External Debt'})
df_ex_debt['Year'] = df_ex_debt['Year'].str.extract('(\d+)').astype(int)
df_ex_debt['External Debt'] = df_ex_debt['External Debt'].replace('..',np.nan).astype(float)
df_ex_debt.set_index(['Year'],append=True,inplace=True)

df_gdp = pd.read_excel(path_to_raw_data+r'gdp_data.xlsx')
df_gdp = df_gdp.iloc[:,2:] # Remove the first 4 columns
df_gdp = df_gdp.set_index(['Country Name','Country Code'],).stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'GDP'})
df_gdp['Year'] = df_gdp['Year'].str.extract('(\d+)').astype(int)
df_gdp['GDP'] = df_gdp['GDP'].replace('..',np.nan).astype(float)
df_gdp.set_index(['Year'],append=True,inplace=True)

df_ex_debt = df_ex_debt.merge(df_gdp,how='left',left_index=True,right_index=True)
df_ex_debt['External Debt to GDP'] = df_ex_debt['External Debt']/df_ex_debt['GDP']

df_ex_debt['Date'] = pd.to_datetime((df_ex_debt.index.get_level_values(2)+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_ex_debt.head()


External Debt           GDP  \
Country Name Country Code Year                                
Afghanistan  AFG          2014   2.020952e+09  2.049713e+10   
                          2015   1.989955e+09  1.913422e+10   
                          2016   1.949506e+09  1.811657e+10   
                          2017   1.981567e+09  1.875346e+10   
                          2018   1.948885e+09  1.805322e+10   

                                External Debt to GDP       Date  
Country Name Country Code Year                                   
Afghanistan  AFG          2014              0.098597 2014-12-31  
                          2015              0.104000 2015-12-31  
                          2016              0.107609 2016-12-31  
                          2017              0.105664 2017-12-31  
                          2018              0.107952 2018-12-31

## OIL PRICE

In [6]:
df_oil = pd.read_excel(path_to_raw_data+r'oil_prices.xlsx', sheet_name='Monthly')
df_oil['observation_date'] = pd.to_datetime(df_oil['observation_date'])
df_oil.set_index('observation_date',inplace=True)
df_oil = df_oil.resample('Y').first() 
df_oil.index = df_oil.index - pd.DateOffset(years=1)
df_oil.rename(columns={'POILBREUSDM':'oil_price'},inplace=True)
df_oil

,oil_price
observation_date,
1989-12-31,20.989130
1990-12-31,23.984783
1991-12-31,18.156522
1992-12-31,17.507143
1993-12-31,14.273810
1994-12-31,16.654091
1995-12-31,18.135217
1996-12-31,23.605652
1997-12-31,15.510455


### Treasury Yields

https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2025

In [7]:
#-------------- Treasury Yields
# find all files in path_to_raw_data/treasury 
files = os.listdir(path_to_raw_data+r'treasury')
df_treasury = pd.DataFrame()

for file in files:
    if 'csv' in file:
        # print(file)
        df = pd.read_csv(path_to_raw_data+r'treasury/'+file)
        df_treasury = pd.concat([df_treasury,df])


# extract the year from the date
df_treasury['Date'] = pd.to_datetime(df_treasury['Date'])
df_treasury['Year'] = df_treasury['Date'].dt.year
df_treasury = df_treasury.sort_values('Date',ascending=True).groupby('Year').last()
# Set date to the last day of the year
for year in df_treasury.index:
    df_treasury.loc[year,'Date'] = datetime(year,12,31)

df_treasury.head(10)


,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Year,,,,,,,,,,,,,,
2017,2017-12-31,1.28,NaN,1.39,NaN,1.53,1.76,1.89,1.98,2.20,2.33,2.40,2.58,2.74
2018,2018-12-31,2.44,2.45,2.45,NaN,2.56,2.63,2.48,2.46,2.51,2.59,2.69,2.87,3.02
2019,2019-12-31,1.48,1.51,1.55,NaN,1.60,1.59,1.58,1.62,1.69,1.83,1.92,2.25,2.39
2020,2020-12-31,0.08,0.08,0.09,NaN,0.09,0.10,0.13,0.17,0.36,0.65,0.93,1.45,1.65
2021,2021-12-31,0.06,0.05,0.06,NaN,0.19,0.39,0.73,0.97,1.26,1.44,1.52,1.94,1.90
2022,2022-12-31,4.12,4.41,4.42,4.69,4.76,4.73,4.41,4.22,3.99,3.96,3.88,4.14,3.97
2023,2023-12-31,5.60,5.59,5.40,5.41,5.26,4.79,4.23,4.01,3.84,3.88,3.88,4.20,4.03
2024,2024-12-31,4.40,4.39,4.37,4.32,4.24,4.16,4.25,4.27,4.38,4.48,4.58,4.86,4.78


### Corporate spread


In [8]:
df_corp_spread = pd.read_excel(path_to_raw_data+r'corporate_spread.xlsx',sheet_name='Sheet1',skiprows=1,index_col=0)
# impute the missing dates in the index 
df_corp_spread.index = pd.to_datetime(df_corp_spread.index)
df_corp_spread = df_corp_spread.resample('Y').last()

df_corp_spread.iloc[-10:]

,VOIG BBB SprdGov,US HY SprdGov,EU Financials SprdGov,EU BBB Yield
2016-12-31,1.23,4.09,1.39,0.8703
2017-12-31,0.86,3.43,0.91,0.7518
2018-12-31,1.52,5.26,1.65,1.2972
2019-12-31,0.93,3.36,0.98,0.5065
2020-12-31,0.92,3.61,0.97,0.2431
2021-12-31,0.95,2.84,1.02,0.5181
2022-12-31,1.67,4.70,2.04,4.3209
2023-12-31,1.38,3.23,1.61,3.5580
2024-12-31,1.02,2.87,1.11,3.1839
2025-12-31,0.90,2.88,0.97,3.1339


### EMBI Yields

In [9]:
#-------------- EMBI with Yields
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    df_embi = pd.read_excel(path_to_raw_data+'EMBI.xlsx',skiprows=4).iloc[:,1:]
    df_embi_coun_code = pd.read_excel(path_to_raw_data+'EMBI.xlsx',sheet_name='risk_region')

df_embi['Year'] = df_embi['Date'].dt.year
for year in df_embi['Year'].unique():
    df_embi.loc[df_embi['Year']==year,'Date'] = datetime(year,12,31)

# DONT USE ANYTHING WHERE COUNTRY_CODE IS NA
df_embi = df_embi.loc[~df_embi['COUNTRY_CODE'].isna()]
# USE MAPPING FROM EXCEL: SAUDI AND CHINA HAS BONDS WITH RISK_REGION OUTSIDE BUT WE CONSIDER THEM STILL AS ASIAN AND MIDDLE EAST
df_embi['RISK_REGION'] = df_embi['COUNTRY_CODE'].map(df_embi_coun_code.set_index('COUNTRY_CODE')['RISK_REGION'])

df_embi.iloc[:5,2:]


,Isin,RISK_REGION,ULTIMATE_PARENT_TICKER,ULT_ISSUER_NAME,SHORT_STD_DESC,Capital_structure,Issue_Date,maturity,rating,NEXT_CALL_PRICE,...,MODEL_OAC,MOD_DUR,OAS,ZV_SPREAD,DTS,DV01_DOLLARS,Avg. yld,COUNTRY_NAME,COUNTRY_CODE,Year
0,XS1819680528,Africa,ANGOL,ANGOLA (REPUBLIC OF),ANGOLA (REPUBLIC OF) RegS,SENIOR,43229,2048-05-08,B3,00:00:00,...,1.200503,8.182979,661.475729,661.475729,5412.841799,1.197873e+06,11.410466,Angola,AGO,2024
1,XS2083302419,Africa,ANGOL,ANGOLA (REPUBLIC OF),ANGOLA (REPUBLIC OF) MTN RegS,SENIOR,43795,2029-11-26,B3,00:00:00,...,0.197349,3.857418,627.536323,627.536323,2420.670026,6.258430e+05,10.537058,Angola,AGO,2024
2,XS1318576086,Africa,ANGOL,ANGOLA (REPUBLIC OF),ANGOLA (REPUBLIC OF) RegS,SENIOR,42320,2025-11-12,B3,00:00:00,...,0.010959,0.806816,482.847475,482.847475,389.569283,7.239836e+04,8.928493,Angola,AGO,2024
3,XS1819680288,Africa,ANGOL,ANGOLA (REPUBLIC OF),ANGOLA (REPUBLIC OF) RegS,SENIOR,43229,2028-05-09,B3,00:00:00,...,0.105400,2.829564,587.384926,587.384926,1662.043207,4.807015e+05,10.126881,Angola,AGO,2024
4,XS2446175577,Africa,ANGOL,ANGOLA (REPUBLIC OF),ANGOLA (REPUBLIC OF) MTN RegS,SENIOR,44665,2032-04-14,B3,00:00:00,...,0.353145,5.020701,645.159659,645.159659,3239.153986,8.149708e+05,10.864871,Angola,AGO,2024


## Rating

In [10]:
rating_dict = {
    # Investment Grade (highest to lowest)
    'Aa2': 20,   # Moody's, equiv to AA
    'AA': 20,    # S&P/Fitch
    'Aa3': 19,   # Moody's, equiv to AA-
    'AA-': 19,   # S&P/Fitch
    'A1': 18,    # Moody's, equiv to A+
    'A+': 18,    # S&P/Fitch
    'A2': 17,    # Moody's, equiv to A
    'A': 17,     # S&P/Fitch (assuming standalone "A" is A)
    'A3': 16,    # Moody's, equiv to A-
    'A-': 16,    # S&P/Fitch
    'Baa1': 15,  # Moody's, equiv to BBB+
    'BBB+': 15,  # S&P/Fitch
    'Baa2': 14,  # Moody's, equiv to BBB
    'BBB': 14,   # S&P/Fitch
    'Baa3': 13,  # Moody's, equiv to BBB-
    'BBB-': 13,  # S&P/Fitch

    # Speculative Grade (highest to lowest)
    'Ba1': 12,   # Moody's, equiv to BB+
    'BB+': 12,   # S&P/Fitch
    'Ba2': 11,   # Moody's, equiv to BB
    'BB': 11,    # S&P/Fitch
    'Ba3': 10,   # Moody's, equiv to BB-
    'BB-': 10,   # S&P/Fitch
    'B1': 9,     # Moody's, equiv to B+
    'B+': 9,     # S&P/Fitch
    'B2': 8,     # Moody's, equiv to B
    'B': 8,      # S&P/Fitch (assuming standalone "B" is B)
    'B3': 7,     # Moody's, equiv to B-
    'B-': 7,     # S&P/Fitch
    'Caa1': 6,   # Moody's, equiv to CCC+
    'CCC+': 6,   # S&P/Fitch
    'Caa2': 5,   # Moody's, equiv to CCC
    'CCC': 5,    # S&P/Fitch
    'Caa3': 4,   # Moody's, equiv to CCC-
    'CCC-': 4,   # S&P/Fitch
    'Ca': 3,     # Moody's, below CCC-
    'C': 2,      # Moody's, lowest (equiv to D in S&P/Fitch)

    # Special case
    0: 0        # Assuming 0 is missing or default, lowest rank
}
rank_to_sp_dict = {
    20: 'AA',    # Highest rank in the provided dict
    19: 'AA-',
    18: 'A+',
    17: 'A',
    16: 'A-',
    15: 'BBB+',
    14: 'BBB',
    13: 'BBB-',
    12: 'BB+',
    11: 'BB',
    10: 'BB-',
    9: 'B+',
    8: 'B',
    7: 'B-',
    6: 'CCC+',
    5: 'CCC',
    4: 'CCC-',
    3: 'CC',    # 'Ca' maps to CC in S&P
    2: 'C',     # 'C' maps to D in S&P
    0: 'D'      # Default/missing rank
}
df_embi['rating_rank'] = df_embi['rating'].map(rating_dict)
df_embi['rating_sp'] = df_embi['rating_rank'].map(rank_to_sp_dict)
df_embi.loc[df_embi['rating']==0,['Date','Isin','COUNTRY_NAME','rating']].sort_values('Date')

,Date,Isin,COUNTRY_NAME,rating
6049,2018-12-31,USP7807HAM71,Venezuela,0
5520,2018-12-31,0,China,0
5531,2018-12-31,XS1646907847,China,0
5536,2018-12-31,USG8201NAG72,China,0
5538,2018-12-31,XS1699084601,China,0
...,...,...,...,...
725,2024-12-31,USP7807HAV70,Venezuela,0
722,2024-12-31,USP7807HAP03,Venezuela,0
719,2024-12-31,XS0294367205,Venezuela,0
789,2024-12-31,XS2328888149,Bahrain,0


### Default 

In [11]:
df_default = pd.read_excel(path_to_raw_data+'Asonuma_Trebesch_DEFAULT_DATABASE.xlsx',sheet_name='DATASET Defaults & Restruct.',usecols='F:H',skiprows=5)
df_default.columns = ['COUNTRY_CODE1','START_DEFAULT_DATE','END_DEFAULT_DATE']

# ---------
# Create an empty list to store the expanded rows
expanded_data = []

# Iterate through each row in the original dataframe
for _, row in df_default.iterrows():
    country = row['COUNTRY_CODE1']

    # Create range of years from start to end (inclusive)
    years = range(int(row['START_DEFAULT_DATE'].year), int(row['END_DEFAULT_DATE'].year) + 1)
    
    # Add each year as a new row with dummy variable = 1
    for year in years:
        expanded_data.append({
            'COUNTRY_CODE': country,
            'Year': year,
            'Default': 1,
            'New_default': 1 if year == row['START_DEFAULT_DATE'].year else 0
        })
    
# Convert the list of dictionaries to a dataframe
df_default = pd.DataFrame(expanded_data)
df_default['Date'] = pd.to_datetime(df_default['Year'].astype(str) + '-12-31')


default_df = pd.DataFrame(index=pd.MultiIndex.from_product([df_default['COUNTRY_CODE'].unique(), pd.date_range(start='1981-12-31', end='2025-12-31', freq='Y')], names=['COUNTRY_CODE1', 'Date']))
default_df = default_df.merge(df_default, how='left',left_index=True, right_on=['COUNTRY_CODE','Date']).set_index(['COUNTRY_CODE','Date'], ).drop(columns=['Year'],)

default_df['years_since_new_default'] = np.nan 
default_df['years_since_in_default'] = np.nan 

for country in df_default['COUNTRY_CODE'].unique():
    df_default_coun = default_df.loc[default_df.index.get_level_values(0) == country]
    for year in df_default_coun.index.get_level_values(1).unique():
        # Get the default year
        default_year = df_default_coun.loc[(df_default_coun.index.get_level_values(1) <= year)&(df_default_coun['Default']==1),]
        if not default_year.empty: 
            default_year = default_year.index.get_level_values(1)[-1]
            default_df.loc[(country, year), 'years_since_in_default'] = int((year-default_year).days/365)

        default_year = df_default_coun.loc[(df_default_coun.index.get_level_values(1) <= year)&(df_default_coun['New_default']==1),]
        if not default_year.empty: 
            default_year = default_year.index.get_level_values(1)[-1]
            # Assign the calculated values to the new columns
            default_df.loc[(country, year), 'years_since_new_default'] = int((year-default_year).days/365)

default_df[['Default','New_default']] = default_df[['Default','New_default']].fillna(0)        

default_df.iloc[8:20] 
# default_dict = df_default.set_index('COUNTRY_CODE1',)['YEARS_SINCE_DEFAULT'].to_dict()



/var/folders/yb/hc1k3b754099lzy1_y8dq0xc0000gn/T/ipykernel_71918/1782357397.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
  default_df.loc[(country, year), 'years_since_in_default'] = int((year-default_year).days/365)
/var/folders/yb/hc1k3b754099lzy1_y8dq0xc0000gn/T/ipykernel_71918/1782357397.py:48: PerformanceWarning: indexing past lexsort depth may impact performance.
  default_df.loc[(country, year), 'years_since_new_default'] = int((year-default_year).days/365)


Default  New_default  years_since_new_default  \
COUNTRY_CODE Date                                                        
ALB          1989-12-31      0.0          0.0                      NaN   
             1990-12-31      0.0          0.0                      NaN   
             1991-12-31      1.0          1.0                      0.0   
             1992-12-31      1.0          0.0                      1.0   
             1993-12-31      1.0          0.0                      2.0   
             1994-12-31      1.0          0.0                      3.0   
             1995-12-31      1.0          0.0                      4.0   
             1996-12-31      0.0          0.0                      5.0   
             1997-12-31      0.0          0.0                      6.0   
             1998-12-31      0.0          0.0                      7.0   
             1999-12-31      0.0          0.0                      8.0   
             2000-12-31      0.0          0.0                      9.0   

                         years_since_in_default  
COUNTRY_CODE Date                                
ALB          1989-12-31                     NaN  
             1990-12-31                     NaN  
             1991-12-31                     0.0  
             1992-12-31                     0.0  
             1993-12-31                     0.0  
             1994-12-31                     0.0  
             1995-12-31                     0.0  
             1996-12-31                     1.0  
             1997-12-31                     2.0  
             1998-12-31                     3.0  
             1999-12-31                     4.0  
             2000-12-31                     5.0

### VIX

In [12]:
df_vix = pd.read_excel(path_to_raw_data+'VIX_History.xlsx',sheet_name='VIX_History')
df_vix['DATE'] = pd.to_datetime(df_vix['DATE'])
df_vix = df_vix.rename(columns={'CLOSE':'VIX'}).set_index('DATE')

# Average over every month 
df_vix = df_vix.resample('M').median()
df_vix.sort_index(inplace=True)
df_vix = df_vix.groupby(df_vix.index.year).last()
df_vix.index = pd.to_datetime(df_vix.index+1,format='%Y')-pd.DateOffset(days=1)
df_vix.head()


,OPEN,HIGH,LOW,VIX
DATE,,,,
1990-12-31,23.94,23.94,23.940,23.940
1991-12-31,17.72,17.72,17.720,17.720
1992-12-31,12.61,12.90,12.145,12.575
1993-12-31,11.11,11.49,10.640,11.030
1994-12-31,12.90,13.48,12.660,12.980


### Terms of Trade

In [13]:
df_tot = pd.read_excel(path_to_raw_data+'Terms of trade.xls',skiprows=3,index_col=[0,1])
df_tot = df_tot.iloc[:,2:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'ToT_Real'}).set_index('Year',append=True)

# df_com_coun_code = pd.read_excel(path_to_raw_data+'Commodity_Terms_of_Trade_Commodity_.xlsx',sheet_name='country_coding',skiprows=3,index_col=0,usecols='D:E')
# df_tot['Country Code'] = df_tot.index.get_level_values(0).map(df_com_coun_code['Country Code'].to_dict())

df_tot['Date'] = pd.to_datetime((df_tot.index.get_level_values(-1).astype(int)+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_tot = df_tot.sort_values(by=['Country Code','Year'])

# CALCULATE THE RUNNING VOLATILITY
df_tot = df_tot.reset_index()  # Reset index to avoid duplicate index issue
df_tot['PCT_TOT_Real'] = df_tot.groupby('Country Code')['ToT_Real'].transform(lambda x: x.pct_change(5))
df_tot['VOL_TOT_Real'] = df_tot.groupby('Country Code')['PCT_TOT_Real'].transform(lambda x: x.rolling(10, min_periods=5).std()) * 100

df_tot['PCT_TOT_Real_3yr'] = df_tot.groupby('Country Code')['ToT_Real'].transform(lambda x: x.pct_change(3))
df_tot['VOL_TOT_Real_3yr'] = df_tot.groupby('Country Code')['PCT_TOT_Real'].transform(lambda x: x.rolling(10, min_periods=5).std()) * 100

df_tot['PCT_TOT_Real_1yr'] = df_tot.groupby('Country Code')['ToT_Real'].transform(lambda x: x.pct_change(1))
df_tot['VOL_TOT_Real_1yr'] = df_tot.groupby('Country Code')['PCT_TOT_Real'].transform(lambda x: x.rolling(10, min_periods=5).std()) * 100

df_tot = df_tot.set_index(['Country Code','Date'])  # Restore the original index if needed
df_tot = df_tot.drop(columns=['Country Name','Year',])
#
df_tot.head(10)



ToT_Real  PCT_TOT_Real  VOL_TOT_Real  \
Country Code Date                                                 
ABW          2000-12-31  103.806927           NaN           NaN   
             2001-12-31   96.453092           NaN           NaN   
             2002-12-31   92.669278           NaN           NaN   
             2003-12-31   99.040741           NaN           NaN   
             2004-12-31  103.446581           NaN           NaN   
             2005-12-31  114.376304      0.101818           NaN   
             2006-12-31  117.517180      0.218387           NaN   
             2007-12-31  118.482620      0.278553           NaN   
             2008-12-31  128.506133      0.297508           NaN   
             2009-12-31   92.695516     -0.103929     16.538146   

                         PCT_TOT_Real_3yr  VOL_TOT_Real_3yr  PCT_TOT_Real_1yr  \
Country Code Date                                                               
ABW          2000-12-31               NaN               NaN               NaN   
             2001-12-31               NaN               NaN         -0.070841   
             2002-12-31               NaN               NaN         -0.039230   
             2003-12-31         -0.045914               NaN          0.068755   
             2004-12-31          0.072507               NaN          0.044485   
             2005-12-31          0.234242               NaN          0.105656   
             2006-12-31          0.186554               NaN          0.027461   
             2007-12-31          0.145351               NaN          0.008215   
             2008-12-31          0.123538               NaN          0.084599   
             2009-12-31         -0.211217         16.538146         -0.278669   

                         VOL_TOT_Real_1yr  
Country Code Date                          
ABW          2000-12-31               NaN  
             2001-12-31               NaN  
             2002-12-31               NaN  
             2003-12-31               NaN  
             2004-12-31               NaN  
             2005-12-31               NaN  
             2006-12-31               NaN  
             2007-12-31               NaN  
             2008-12-31               NaN  
             2009-12-31         16.538146

### Commodity Index

In [14]:
# df_com_imp = pd.read_excel(path_to_raw_data+'Commodity_Terms_of_Trade_Commodity_.xlsx',skiprows=1,index_col=0)
df_com_exp = pd.read_excel(path_to_raw_data+'COM_TOT_EXP.xlsx',skiprows=1,index_col=0)
df_com_coun_code = pd.read_excel(path_to_raw_data+'Commodity_Terms_of_Trade_Commodity_.xlsx',sheet_name='country_coding',skiprows=3,index_col=0,usecols='D:E')

# df_com_imp = df_com_imp.stack().reset_index(level=[-1]).rename(columns={'level_1':'Year',0:'imp'}).set_index('Year',append=True)
df_com_exp = df_com_exp.stack().reset_index(level=[-1]).rename(columns={'level_1':'Year',0:'exp'}).set_index('Year',append=True)

# 
df_com = df_com_exp.reset_index(drop=False).rename(columns={'exp':'com_idx','level_0':'Country Name'}).set_index('Country Name')

df_com['Country Code'] = df_com.index.map(df_com_coun_code['Country Code'].to_dict())
df_com['Date'] = pd.to_datetime((df_com['Year'].astype(int)+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_com = df_com.sort_values(by=['Country Code','Year'])

# # # CALCULATE THE RUNNING VOLATILITY

idx_name = df_com.index.name if df_com.index.name is not None else 'index'
df_com = df_com.reset_index()  # Reset index to avoid duplicate index issue
df_com['com_idx_chg_5yr'] = df_com.groupby('Country Code')['com_idx'].transform(lambda x: x.pct_change(5))
df_com['com_idx_vol_10yr'] = df_com.groupby('Country Code')['com_idx_chg_5yr'].transform(lambda x: x.rolling(10, min_periods=5).std())

df_com = df_com.set_index(['Country Code','Date']).drop(columns=['Country Name','Year'])  # Restore the original index if needed

#
df_com.head(10)

com_idx  com_idx_chg_5yr  com_idx_vol_10yr
Country Code Date                                                     
AGO          2007-12-31   90.974739              NaN               NaN
             2008-12-31   99.627274              NaN               NaN
             2009-12-31   84.924828              NaN               NaN
             2010-12-31   92.445312              NaN               NaN
             2011-12-31  100.594528              NaN               NaN
             2012-12-31  100.000000         0.099206               NaN
             2013-12-31  100.768356         0.011454               NaN
             2014-12-31   98.168411         0.155945               NaN
             2015-12-31   78.654137        -0.149182               NaN
             2016-12-31   75.567116        -0.248795          0.169554

### Gold reserves to GDP

In [33]:
df_gold = pd.read_excel(path_to_raw_data+'gold_reserves.xlsx')

df_gold = df_gold.set_index(['Country Name','Country Code'],).iloc[:,2:].stack().reset_index(level=[-1]).rename(columns={'level_2':'Year',0:'Gold Reserves'})
df_gold['Year'] = df_gold['Year'].str.extract('(\d+)').astype(int)
df_gold.set_index(['Year'],append=True,inplace=True)
df_gold['Gold Reserves'] = df_gold['Gold Reserves'].replace('..',np.nan).astype(float)

df_gold = df_gold.merge(df_gdp,how='left',left_index=True,right_index=True)
df_gold['Gold_to_GDP'] = df_gold['Gold Reserves']/df_ex_debt['GDP']

df_gold['Date'] = pd.to_datetime((df_gold.index.get_level_values(2)+1).astype(str),format='%Y')-pd.DateOffset(days=1)
df_gold.tail()

df_test = df_gold.loc[(df_gold.index.get_level_values(1).isin(df_embi['COUNTRY_CODE'].unique()))&(df_gold.index.get_level_values(2).isin([2009, 2021]))]

# # df_gold
# df = df.merge(df_gold.reset_index().set_index(['Country Code','Date'])['Gold_to_GDP'], left_on=['COUNTRY_CODE','Date'],right_index=True,how='left')
# df.loc[(df['Gold_to_GDP'].isna())&(df['Date']<=datetime(2024,1,1)),['COUNTRY_CODE','Date']]



In [37]:
df_test['Gold_to_GDP'] = df_test['Gold Reserves']/df_test['GDP']
df_test.groupby(level=2)['Gold_to_GDP'].describe()

/var/folders/yb/hc1k3b754099lzy1_y8dq0xc0000gn/T/ipykernel_71918/469323004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Gold_to_GDP'] = df_test['Gold Reserves']/df_test['GDP']


,count,mean,std,min,25%,50%,75%,max
Year,,,,,,,,
2009,75.0,0.220551,0.173608,0.020188,0.126909,0.177307,0.242369,1.105432
2021,73.0,0.229729,0.184915,0.035402,0.124703,0.201588,0.281389,1.523401


# Merge Data

In [ ]:
default_df

In [ ]:
df_embi

In [ ]:
# Merge EMBI with treasury yields
df = df_embi.merge(df_treasury, left_on='Date',right_on='Date',how='left')
df = df.merge(df_ex_debt.reset_index()[['Country Code','Date','External Debt to GDP']], left_on=['Date','COUNTRY_CODE'],right_on=['Date','Country Code'],how='left')

# df_gold
df = df.merge(df_gold.reset_index().set_index(['Country Code','Date'])['Gold_to_GDP'], left_on=['COUNTRY_CODE','Date'],right_index=True,how='left')
df.loc[(df['Gold_to_GDP'].isna())&(df['Date']<=datetime(2024,1,1)),['COUNTRY_CODE','Date']]
# df_gold.loc[(df_gold.index.get_level_values(1)=='ETH')&(df_gold['Date']==datetime(2023,12,31)) ]
print('Theses Country codes are not in Gold reserves', df.loc[(df['Gold_to_GDP'].isna())&(df['Date']<=datetime(2024,1,1)),'COUNTRY_CODE'].sort_values().unique())

# VIX
df = df.merge(df_vix['VIX'],left_on='Date',right_index=True,how='left')

# Default
df = df.merge(default_df, left_on=['COUNTRY_CODE','Date'], right_index=True, how='left')

# # # Commodity
# df = df.merge(df_com.drop_duplicates(subset=['Country Code', 'Date'], ).set_index(['Country Code','Date'])[['ToT', 'VOL_TOT', 'PCT_TOT']],left_on=['COUNTRY_CODE','Date'],right_index=True,how='left')

# Terms of Trade
df = df.merge(df_tot, left_on=['COUNTRY_CODE','Date'],right_index=True, how='left')

# BBB spread 
df = df.merge(df_corp_spread, left_on='Date',right_index=True,how='left')

# Oil
df = df.merge(df_oil, left_on='Date',right_index=True,how='left')

df = df.merge(df_com, left_on=['Country Code','Date'],right_index=True,how='left')

df['Date'] = pd.to_datetime(df['Date'])
df.iloc[:,-40:-20]



In [ ]:
df['slope'] = df['10 Yr'] - df['2 Yr']
df['treasury_yield'] = np.nan
df['nss_model'] = None
df['years_to_maturity'] = (pd.to_datetime(df['maturity']) - pd.to_datetime(df['Date'])).dt.days/365

t_str = ['1 Mo', '2 Mo', '3 Mo', '4 Mo','6 Mo', '1 Yr', '2 Yr', '3 Yr', '5 Yr', '7 Yr','10 Yr','20 Yr','30 Yr']
t = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])


y = []
for i, d in enumerate(df['Date'].unique()):
    sliced_df = df.loc[df['Date'] == d, :]
    y = sliced_df[['1 Mo', '2 Mo', '3 Mo', '4 Mo','6 Mo', '1 Yr', '2 Yr', '3 Yr', '5 Yr', '7 Yr','10 Yr','20 Yr','30 Yr']].iloc[0,:].values
    # remove nan
    y_new = y[~np.isnan(y)]
    t_new = t[~np.isnan(y)]
    # calibrate nelson siegel
    nss_model = calibrate_nss_ols(t_new, y_new)
    # add to dataframe
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        df.loc[df['Date'] == d, 'treasury_yield']  = nss_model[0].zero(sliced_df['years_to_maturity'].values)
    warnings.simplefilter('default')
    

# Calculate the spread
df['spread'] = df['Avg. yld'] - df['treasury_yield']

# Remove long bonds
# print(df.loc[df['years_to_maturity']>=30,['Isin','maturity','COUNTRY_NAME']].drop_duplicates().shape[0])
df = df.loc[df['years_to_maturity']<30]

df = df.dropna(subset=['COUNTRY_CODE']).query('NOMINAL_YIELD != 0')
df = df.loc[~df['Isin'].isin(['XS1391003446', 0,])]

In [ ]:
df['constant'] = 1 
df[['Avg. yld','treasury_yield','spread']]

In [ ]:
df.to_excel(path_to_data+'embi_regression_data.xlsx',index=False)

## Additional notes: 
- Consider using Total debt rather than external debt to get data for more countries
- Maybe use IG spread to get some sort of risc appettite measure